## Toronto Cruster Analysis

#scrape toronto data from wiki

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [2]:
df = df[0]

In [3]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#Cleanning 'Not Assigned' Data

In [4]:
newdf = df

In [5]:
newdf = newdf[newdf.Borough !='Not assigned']

In [6]:
newdf.Neighbourhood.size

212

In [7]:
for i in range(newdf.Neighbourhood.size):
    if newdf['Neighbourhood'].values[i] == 'Not assigned':
        newdf['Neighbourhood'].values[i] = newdf['Borough'].values[i]

In [8]:
newdf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#group Postcode and join Neighbourhood

In [9]:
Mdf = newdf.groupby(['Postcode','Borough'])['Neighbourhood'].agg(', '.join).reset_index()

In [10]:
Mdf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
Mdf.shape

(103, 3)

#Read Toronto Geo CSV file

In [12]:
rawdf = pd.read_csv('http://cocl.us/Geospatial_data')
rawdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#Merge data frame

In [13]:
Mdf['Latitude'] = Mdf.Postcode.map(rawdf.set_index('Postal Code')['Latitude'].to_dict())
Mdf['Longitude'] = Mdf.Postcode.map(rawdf.set_index('Postal Code')['Longitude'].to_dict())
Mdf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#import liblary and cal Toronto coordinate

In [14]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#Create map

In [16]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Mdf['Latitude'], Mdf['Longitude'], Mdf['Borough'], Mdf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
Mdf.groupby(['Borough']).count()

,Postcode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [18]:
Mdf.shape

(103, 5)

In [19]:
intown_list = ['Central Toronto', 'Downtown Toronto', 'East Toronto', 'West Toronto']

intown = Mdf[Mdf['Borough'].isin (intown_list)].reset_index()
intown.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [20]:
# create map of toronto using latitude and longitude values
map_intown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(intown['Latitude'], intown['Longitude'], intown['Borough'], intown['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_intown)  
    
map_intown

In [21]:
intown.shape

(38, 6)

#get data from Foursquare

In [22]:
CLIENT_ID = 'BBLJWEETWIJCLJQVRRV5IUJY1N3EVSBRAOBNNFFSW45LPM5F' # your Foursquare ID
CLIENT_SECRET = 'B1VTLXFXHUBZZ0FZLFCLIDQ2RKRTYAVTKZYAWVWNY5C2RNGW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [23]:
#nearby function

LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#run function for intown Toronto

In [24]:
intown_venues = getNearbyVenues(names=intown['Neighbourhood'],
                                   latitudes=intown['Latitude'],
                                   longitudes=intown['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [25]:
#check result dataframe
print(intown_venues.shape)
intown_venues.head()

(1692, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [26]:
intown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


In [27]:
print('There are {} uniques categories.'.format(len(intown_venues['Venue Category'].unique())))

There are 234 uniques categories.


Analyze each Neighbourhood 

In [28]:
# one hot encoding
intown_onehot = pd.get_dummies(intown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
intown_onehot['Neighbourhood'] = intown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [intown_onehot.columns[-1]] + list(intown_onehot.columns[:-1])
intown_onehot = intown_onehot[fixed_columns]

#intown_venues.head()
intown_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
intown_grouped = intown_onehot.groupby('Neighbourhood').mean().reset_index()
intown_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.000000,0.000000,0.011905,0.000000,0.000000,0.011905,0.000000,0.000000,0.00,0.011905
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.050000,0.000000,0.030000,0.010000,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011905,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.000000,0.000000,0.000000,0.011905,0.011905,0.000000,0.011905,0.011905,0.00,0.011905


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creat new dataframe shows top 10 common venues

In [84]:
import numpy as np
import pandas as pd

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = intown_grouped['Neighbourhood']

for ind in np.arange(intown_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(intown_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,American Restaurant,Gym,Asian Restaurant,Burger Joint,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Café,Italian Restaurant,Steakhouse,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Grocery Store,Gym,Performing Arts Venue,Pet Store,Bar,Stadium,Convenience Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Plane


## Cluster of neighbourhood

In [85]:
# set number of clusters
kclusters = 20

grouped_clustering = intown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 0, 15, 19, 14,  4, 15, 15,  0, 12,  0], dtype=int32)

In [86]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

intown_merged = intown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
intown_merged = intown_merged.join(venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

intown_merged.tail() # check the last columns!

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,78,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,19,Breakfast Spot,Café,Coffee Shop,Grocery Store,Gym,Performing Arts Venue,Pet Store,Bar,Stadium,Convenience Store
34,82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,9,Mexican Restaurant,Café,Thai Restaurant,Bar,Italian Restaurant,Bakery,Fried Chicken Joint,Cajun / Creole Restaurant,Furniture / Home Store,Speakeasy
35,83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,11,Breakfast Spot,Gift Shop,Bookstore,Dessert Shop,Burger Joint,Eastern European Restaurant,Bar,Bank,Movie Theater,Restaurant
36,84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Tea Room,Fish & Chips Shop,Food & Drink Shop,Bar,Food
37,87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,14,Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(intown_merged['Latitude'], intown_merged['Longitude'], intown_merged['Neighbourhood'], intown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Cluster 1

In [90]:
intown_merged.loc[intown_merged['Cluster Labels'] == 0, intown_merged.columns[[1] + list(range(5, intown_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M4L,-79.315572,0,Park,Pet Store,Sushi Restaurant,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Movie Theater
7,M4S,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Coffee Shop,Italian Restaurant,Pharmacy,Sushi Restaurant,Gym,Brewery
12,M4Y,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Gastropub,Fast Food Restaurant,Mediterranean Restaurant
14,M5B,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Restaurant,Tea Room,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place
18,M5H,-79.384568,0,Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,American Restaurant,Gym,Asian Restaurant,Burger Joint,Bakery
25,M5S,-79.400049,0,Café,Coffee Shop,Bar,Japanese Restaurant,Restaurant,Bookstore,Bakery,Dessert Shop,Chinese Restaurant,Pub
26,M5T,-79.400049,0,Bar,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Arts & Crafts Store
32,M6J,-79.419750,0,Bar,Coffee Shop,Asian Restaurant,Men's Store,Pizza Place,Restaurant,Bakery,Cocktail Bar,Café,Vietnamese Restaurant
36,M6S,-79.484450,0,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Tea Room,Fish & Chips Shop,Food & Drink Shop,Bar,Food


Cluster 2

In [91]:
intown_merged.loc[intown_merged['Cluster Labels'] == 15, intown_merged.columns[[1] + list(range(5, intown_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M4M,-79.340923,15,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Gastropub,Chinese Restaurant,Fish Market,Juice Bar,Bookstore
11,M4X,-79.367675,15,Coffee Shop,Restaurant,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Park,Pharmacy,Breakfast Spot
13,M5A,-79.360636,15,Coffee Shop,Pub,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center,Yoga Studio
15,M5C,-79.375418,15,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Italian Restaurant,Cosmetics Shop,Cocktail Bar,Park,Clothing Store
16,M5E,-79.373306,15,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Café,Italian Restaurant,Steakhouse,Farmers Market
17,M5G,-79.387383,15,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Bar,Chinese Restaurant,Thai Restaurant,Spa,Sushi Restaurant
19,M5J,-79.381752,15,Coffee Shop,Aquarium,Hotel,Pizza Place,Italian Restaurant,Café,Brewery,Restaurant,Scenic Lookout,Bakery
20,M5K,-79.381576,15,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Burger Joint,Steakhouse,Gym
21,M5L,-79.379817,15,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Steakhouse,Bakery
28,M5W,-79.374846,15,Coffee Shop,Restaurant,Café,Hotel,Seafood Restaurant,Fast Food Restaurant,Beer Bar,Italian Restaurant,Cocktail Bar,Art Gallery


Cluster 3

In [99]:
intown_merged.loc[intown_merged['Cluster Labels'] == 4, intown_merged.columns[[1] + list(range(5, intown_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,-79.39442,4,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Plane


## Conclusion
 - We need to increase number of cluster to see the differences due to the most area is full with coffee shops, cafes and restaurants
 - In the center of Toronto is full of Coffee shops and cafes (cluster 0 and cluster 15)
 - outside of Toronto we can have parks, playgrounds (cluster 6 and cluster 1), airport services (cluster 4)